In [ ]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Load and preprocess dataset (adjust dataset path accordingly)
def load_dataset(base_path, target_size=(128, 128)):
    classes = ['Non_Demented', 'Very_Mild_Demented', 'Mild_Demented', 'Moderate_Demented']
    images, labels = [], []
    for idx, label in enumerate(classes):
        class_path = os.path.join(base_path, label)
        for file in os.listdir(class_path):
            image = cv2.imread(os.path.join(class_path, file))
            if image is not None:
                image = cv2.resize(image, target_size)
                images.append(image)
                labels.append(idx)
    return np.array(images), np.array(labels)

# Assuming the dataset is stored in /content/dataset
base_path = '/content/dataset'
images, labels = load_dataset(base_path)

# Normalize the data
images = images / 255.0

# Split the data into training, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Define a simple CNN model
def build_cnn(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 classes
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the model
def train_and_evaluate(x_train, x_val, x_test, y_train, y_val, y_test, input_shape):
    model = build_cnn(input_shape)
    model.summary()
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"Test Accuracy: {test_acc:.4f}")
    return history, test_acc

# Run training and evaluation on Original Images
print("Training on Original Images...")
history_original, acc_original = train_and_evaluate(
    x_train, x_val, x_test, y_train, y_val, y_test, input_shape=(128, 128, 3)
)

print(f"Original Images Test Accuracy: {acc_original:.4f}")
